In [1]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

c:\Users\pedro\Desktop\CrewAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Carregar variável de ambiente
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [10]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") # Garanta que sua variável de ambiente se chama GOOGLE_API_KEY
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Chave da API do Google configurada com sucesso!")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente.")
    # GOOGLE_API_KEY = input("Por favor, insira sua Chave da API do Google (Gemini): ")
    # genai.configure(api_key=GOOGLE_API_KEY)

Chave da API do Google configurada com sucesso!


In [11]:
# Função para criar agentes simulando comportamentos
def gerar_resposta_gemini(prompt):
    model = genai.GenerativeModel("models/gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text.strip()

In [6]:
def gerar_resposta_gemini(prompt, nome_modelo="gemini-1.5-flash"): # Ou o modelo que preferir
    if not GOOGLE_API_KEY:
        return "Erro: Chave da API do Google não configurada."
    try:
        model = genai.GenerativeModel(nome_modelo)
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Erro ao gerar resposta do Gemini: {e}

In [13]:
def agente_coletor_noticias_acoes(nome_acao_ou_ticker):
    print(f"\n🔎 Buscando notícias para '{nome_acao_ou_ticker}'...")

    try:
        print("Usando texto de notícias de exemplo (simulação).")
        noticias_exemplo = f"""
        Recente alta nas ações de {nome_acao_ou_ticker} impulsionada por resultados trimestrais acima do esperado.
        A {nome_acao_ou_ticker} anunciou um novo plano de investimento em tecnologia sustentável, visando reduzir emissões.
        Analistas de mercado estão com opiniões divididas sobre {nome_acao_ou_ticker}, com alguns apontando potencial de crescimento e outros alertando para riscos setoriais.
        A empresa {nome_acao_ou_ticker} divulgou comunicado sobre mudanças na diretoria executiva.
        O setor de {nome_acao_ou_ticker} enfrenta desafios com a volatilidade cambial e aumento dos custos de matéria-prima.
        """
        return noticias_exemplo.strip()

    except Exception as e:
        print(f"Erro na busca simulada de notícias (usando exemplo): {e}")
        return f"Não foi possível buscar notícias para {nome_acao_ou_ticker} devido a um erro. Exemplo: A empresa XPTO anunciou um novo produto."


# Agente: Resumidor de Notícias de Ações
def agente_resumidor_noticias(texto_noticias, nome_acao):
    if not texto_noticias or "Não foi possível buscar notícias" in texto_noticias:
        return "Sem notícias para resumir."
    prompt = f"""
    Você é um analista financeiro conciso.
    Com base no compilado de notícias abaixo sobre a empresa/ação '{nome_acao}',
    elabore um resumo dos pontos mais importantes e relevantes para um potencial investidor.
    Destaque fatos, números chave (se houver) e principais acontecimentos.
    Evite especulações não fundamentadas diretamente nas notícias fornecidas.

    NOTÍCIAS COMPILADAS:
    ---
    {texto_noticias}
    ---

    RESUMO EXECUTIVO DAS NOTÍCIAS:
    """
    return gerar_resposta_gemini(prompt)

# Agente: Analisador de Sentimento de Notícias
def agente_analisador_sentimento(texto_noticias, nome_acao):
    if not texto_noticias or "Não foi possível buscar notícias" in texto_noticias:
        return "Sem notícias para analisar sentimento."
    prompt = f"""
    Você é um especialista em análise de sentimento de textos financeiros.
    Leia o compilado de notícias abaixo sobre a empresa/ação '{nome_acao}'.
    Qual é o sentimento geral predominante expresso nessas notícias em relação à empresa/ação?
    Classifique como: Positivo, Negativo ou Neutro/Misto.
    Forneça uma breve justificativa para sua classificação, baseando-se nos principais pontos das notícias.

    NOTÍCIAS COMPILADAS:
    ---
    {texto_noticias}
    ---

    ANÁLISE DE SENTIMENTO:
    Sentimento Predominante: [Positivo/Negativo/Neutro/Misto]
    Justificativa:
    """
    return gerar_resposta_gemini(prompt)

# Agente: Descritor de Empresa
def agente_descritor_empresa(nome_acao_ou_ticker):
    print(f"\nℹ️ Buscando descrição para '{nome_acao_ou_ticker}'...")

    descricao_bruta_exemplo = f"Informações encontradas sobre {nome_acao_ou_ticker} indicam que é uma empresa atuante no setor de [preencher setor], com foco em [preencher produtos/serviços principais]. Sua história remonta a [preencher ano, se souber] e tem como principais concorrentes [preencher se souber]."
    print("Usando texto de descrição de exemplo (simulação).")


    prompt = f"""
    Você é um redator de perfis corporativos.
    Com base na informação bruta fornecida abaixo sobre '{nome_acao_ou_ticker}', crie uma descrição clara e concisa sobre
    o que a empresa faz, seu principal setor de atuação e, se possível, um breve diferencial ou foco.

    INFORMAÇÃO BRUTA:
    ---
    {descricao_bruta_exemplo}
    ---

    PERFIL DA EMPRESA {nome_acao_ou_ticker.upper()}:
    """
    return gerar_resposta_gemini(prompt)

# --- EXECUÇÃO DO ASSISTENTE DE AÇÕES ---

if GOOGLE_API_KEY: # Só executa se a API estiver configurada
    acao_desejada = input("Digite o nome ou ticker da ação para consulta (ex: PETR4, MGLU3, Apple): ")

    if acao_desejada:
        # 1. Coletar Notícias
        noticias = agente_coletor_noticias_acoes(acao_desejada)
        # print(f"\n📰 Notícias Coletadas (Bruto/Simulado):\n{noticias}") # Para debug

        # 2. Resumir Notícias
        resumo = agente_resumidor_noticias(noticias, acao_desejada)

        # 3. Analisar Sentimento
        sentimento = agente_analisador_sentimento(noticias, acao_desejada)

        # 4. Descrever Empresa
        descricao_empresa = agente_descritor_empresa(acao_desejada)

        # Apresentar Resultados Consolidados
        print(f"\n\n--- 📈 Análise Consolidada para: {acao_desejada.upper()} ---")
        print("="*70)

        print("\n📝 PERFIL DA EMPRESA:")
        print(descricao_empresa)
        print("-"*70)

        print("\n📰 RESUMO DAS NOTÍCIAS RECENTES (Baseado em busca simulada/genérica):")
        print(resumo)
        print("-"*70)

        print("\n📊 ANÁLISE DE SENTIMENTO DAS NOTÍCIAS (Baseado em busca simulada/genérica):")
        print(sentimento)
        print("="*70)
        print("\nLembrete: As informações acima são geradas por IA e não constituem aconselhamento financeiro.")

    else:
        print("Nenhuma ação informada.")
else:
    print("\nO assistente não pode ser executado pois a chave da API do Google não foi configurada.")


🔎 Buscando notícias para 'VALE3'...
Usando texto de notícias de exemplo (simulação).

ℹ️ Buscando descrição para 'VALE3'...
Usando texto de descrição de exemplo (simulação).


--- 📈 Análise Consolidada para: VALE3 ---

📝 PERFIL DA EMPRESA:
PERFIL DA EMPRESA VALE3:

A VALE3 é uma empresa líder global no setor de mineração, com foco na produção e comercialização de minério de ferro, pelotas de minério de ferro, níquel e outros metais básicos.  Sua história se inicia em 1942 (como Companhia Vale do Rio Doce), e ao longo de sua trajetória consolidou-se como uma das maiores mineradoras do mundo.  A empresa se destaca por sua integração vertical, abrangendo desde a extração de minério até o beneficiamento e logística, e por seu compromisso com práticas sustentáveis e inovadoras na indústria.
----------------------------------------------------------------------

📰 RESUMO DAS NOTÍCIAS RECENTES (Baseado em busca simulada/genérica):
RESUMO EXECUTIVO DAS NOTÍCIAS:

VALE3 apresentou resultados t